In [ ]:
import psycopg2
import psycopg2.extras
import getpass
from IPython.display import display
import ipywidgets

In [ ]:
def hmstodeg( h, m, s):
    return 15 * ( h + m/60. + s/3600.)

def dmstodeg( d, m, s ):
    return d + m/60. + s/3600.

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
# Find all candidates within 5" of 10:02:15.36, +03:12:37.1
ra = hmstodeg( 10, 2, 15.36 )
dec = dmstodeg( 3,12, 37.1 )
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )
query = ( 'SELECT c.id,c.ra,c.dec,COUNT(o.id) as numobj FROM candidates c '
          'LEFT JOIN objects o ON c.id=o.candidate_id '
          'WHERE q3c_radial_query(c.ra,c.dec,%s,%s,%s) ' 
          'GROUP BY c.id,c.ra,c.dec' )
cursor.execute( query, ( ra, dec, 5./3600.) )
print( f'{"Candidate":10s}  {"    RA":10s}  {"    dec":10s}  Num. objects' )
print( '================================================' )
for row in cursor:
    print( f'{row["id"]:10s}  {row["ra"]:10.5f}  {row["dec"]:10.5f}  {row["numobj"]}' )
cursor.close()
    

In [ ]:
# Based on the results above, I want to look at candidate DC21jay
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )
query = ( 'SELECT e.mjd,e.filename,e.filter,c.sci_jpeg,c.ref_jpeg,c.diff_jpeg FROM cutouts c '
          'INNER JOIN objects o ON c.object_id=o.id '
          'INNER JOIN candidates cand ON o.candidate_id=cand.id '
          'INNER JOIN subtractions s ON o.subtraction_id=s.id '
          'INNER JOIN exposures e ON s.exposure_id=e.id '
          'WHERE cand.id=%s '
          'ORDER BY e.mjd ')
cursor.execute( query, ( "DC21jay", ) )
for row in cursor:
    label = ipywidgets.HTML( f'<b>{row["filename"]}</b><br><b>Filter:</b> {row["filter"]}' )
    newimg = ipywidgets.Image( value=bytes(row["sci_jpeg"]), width=102, height=102, format='jpeg' )
    refimg = ipywidgets.Image( value=bytes(row["ref_jpeg"]), width=102, height=102, format='jpeg' )
    subimg = ipywidgets.Image( value=bytes(row["diff_jpeg"]), width=102, height=102, format='jpeg' )
    box = ipywidgets.HBox( [ newimg, refimg, subimg, label ] )
    display( box )
cursor.close()
print( "Did" )